In [1]:
import pandas as pd
import numpy as np
import sys

sys.path.append("..")

# df = pd.read_csv('../weekly opt selling positional JJMS/rbw_nifty_sensex_combined.csv')
df = pd.read_csv('NLC - JJ OS/Lobster_1_30_w_hedges.csv')
df.tail()

,Day,Strike,Expiry,DTE,Option Type,Entry Price,Initial SL,Exit Price,Exit Time,Max Price After Entry,...,ROI%,Trade Year,Entry Time,Hedge Strike,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Hedge ROI%,Total ROI%
2971,2024-12-27,23550,2025-01-02,6,PE,NaN,NaN,NaN,NaN,NaN,...,NaN,2024,09:15:00,0,NaN,NaN,NaN,NaN,0.0,NaN
2972,2024-12-30,24050,2025-01-02,3,CE,NaN,NaN,NaN,NaN,NaN,...,NaN,2024,09:15:00,0,NaN,NaN,NaN,NaN,0.0,NaN
2973,2024-12-30,23550,2025-01-02,3,PE,NaN,NaN,NaN,NaN,NaN,...,NaN,2024,09:15:00,0,NaN,NaN,NaN,NaN,0.0,NaN
2974,2024-12-31,23800,2025-01-02,2,CE,NaN,NaN,NaN,NaN,NaN,...,NaN,2024,09:15:00,0,NaN,NaN,NaN,NaN,0.0,NaN
2975,2024-12-31,23300,2025-01-02,2,PE,NaN,NaN,NaN,NaN,NaN,...,NaN,2024,09:15:00,0,NaN,NaN,NaN,NaN,0.0,NaN


In [5]:
import datetime as dt
df['ROI% PF'] = df['Total ROI%']

# Convert 'Entry Time' and 'Exit Time' to datetime
# Ensure 'Day' is a datetime object
df['Day'] = pd.to_datetime(df['Day'])

# Combine 'Day' and 'Entry Time' into a single datetime column
df['DATETIME'] = pd.to_datetime(df['Day'].dt.date.astype(str) + ' ' + df['Entry Time'].astype(str))

df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['Exit Time'] = pd.to_datetime(df['Exit Time'])

# Calculate cumulative ROI
df['Cumulative ROI'] = df['ROI% PF'].cumsum()

# Calculate the running maximum of the cumulative ROI
df['Cumulative Max'] = df['Cumulative ROI'].cummax()

# Identify drawdown: when cumulative ROI is below the cumulative max
df['In Drawdown'] = df['Cumulative ROI'] < df['Cumulative Max']

# Label drawdown cycles
df['Drawdown Cycle'] = (df['In Drawdown'] & ~df['In Drawdown'].shift(fill_value=False)).cumsum()

# Calculate time in drawdown for each cycle
drawdown_stats = (
    df[df['In Drawdown']]
    .groupby('Drawdown Cycle')
    .agg(
        Entry_Time=('DATETIME', 'first'),  # First entry time in the drawdown cycle
        Exit_Time=('Exit Time', 'last'),    # Last exit time in the drawdown cycle
    )
    .reset_index()
)

# Calculate the duration in days for each drawdown cycle
drawdown_stats['Time_in_Drawdown (Days)'] = (
    (drawdown_stats['Exit_Time'] - drawdown_stats['Entry_Time']).dt.days
)

# Sort by the longest drawdown durations and take the top 5
top_drawdowns = drawdown_stats.nlargest(5, 'Time_in_Drawdown (Days)')

# Output the top drawdowns
print("Top 5 Time in Drawdown Periods (in Days):")
print(pd.DataFrame(top_drawdowns).to_string())

# Merge results with the original DataFrame for context if needed
# df = df.merge(drawdown_stats, on='Drawdown Cycle', how='left')

# print("\nAnnotated DataFrame:")
# print(df.to_string())

Top 5 Time in Drawdown Periods (in Days):
     Drawdown Cycle          Entry_Time           Exit_Time  Time_in_Drawdown (Days)
33               34 2019-07-05 09:15:00 2019-09-20 10:45:00                       77
316             317 2023-09-18 09:15:00 2023-11-10 12:44:00                       53
298             299 2023-04-17 09:15:00 2023-06-05 15:30:00                       49
236             237 2022-04-05 09:15:00 2022-05-20 15:30:00                       45
245             246 2022-06-30 09:15:00 2022-07-28 10:13:00                       28


# Lobster